In [13]:
import mlflow
import mlflow.sklearn
import json
import os
import tempfile
import matplotlib.pyplot as plot
import seaborn as sb 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, classification_report, plot_confusion_matrix
from mlflow.models.signature import infer_signature
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import numpy as np

mlflow.set_experiment("SKLearn Sentiment")
count=CountVectorizer()
data=pd.read_csv("Tweets.csv")
data.head()

data['sentiment']=data['airline_sentiment'].apply(lambda x: 0 if x=='negative' else 1)
data['sentiment'] = data['sentiment'].astype(np.float64)
features = data.text.values
labels = data.sentiment.values
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,sentiment
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),1.0
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1.0
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),1.0
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0.0
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0.0


In [14]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))
    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 
    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [15]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(processed_features).toarray()



In [21]:
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [26]:
from sklearn.ensemble import RandomForestClassifier
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")


In [23]:
with mlflow.start_run(run_name="TFIDF + Random Forest 2"):
    
    text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
    text_classifier.fit(X_train, y_train)


    y_pred = text_classifier.predict(X_test)
    
    
    signature = infer_signature(X_test, y_test)
    
    mlflow.log_metric('f1', f1_score(y_test, y_pred, average='micro'))
    mlflow.sklearn.log_model(text_classifier, "model", signature=signature)
    
    with tempfile.TemporaryDirectory() as tmpdir:
        
        report = classification_report(y_test, y_pred, output_dict=True)
        
        with open(os.path.join(tmpdir, "classification_report.json"),'w') as f:
            json.dump(report, f, indent=2)
            
        mlflow.log_artifacts(tmpdir, "reporting")

C:\Users\kumar\.conda\envs\mlflow\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
https://medium.com/analytics-vidhya/integrate-mlflow-model-logging-to-scikit-learn-pipeline-6f74e5c601c3

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,sentiment
8224,568588936852799488,positive,0.6830,NaN,0.0000,Delta,NaN,AmazingLaisha,NaN,0,@JetBlue yes thankfully! Catering just got her...,NaN,2015-02-19 17:52:21 -0800,✈️FL/NJ/NYC,Eastern Time (US & Canada),1.0
1665,569631027385245696,negative,1.0000,Lost Luggage,1.0000,United,NaN,liveitup09,NaN,1,@united yes I have &amp; they're unsure when i...,NaN,2015-02-22 14:53:14 -0800,NaN,Atlantic Time (Canada),0.0
2157,569336402401472512,negative,0.6988,Cancelled Flight,0.3695,United,NaN,Colettod,NaN,0,@united Wife and I have two new destinations a...,NaN,2015-02-21 19:22:30 -0800,"Ottawa, ON",Eastern Time (US & Canada),0.0
2464,569188938012880896,negative,1.0000,Lost Luggage,1.0000,United,NaN,NewsLadiesNeed,NaN,0,@united Why do I still not have my bags? They ...,"[33.89638659, -84.33108377]",2015-02-21 09:36:32 -0800,Atlanta,Eastern Time (US & Canada),0.0
12206,570263732099649537,neutral,1.0000,NaN,NaN,American,NaN,towbinator,NaN,0,@AmericanAir @dfwairport That's DFW!? OMG! Yea...,NaN,2015-02-24 08:47:23 -0800,United States of America,Eastern Time (US & Canada),1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,569934458364813313,neutral,1.0000,NaN,NaN,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN,1.0
3264,568564006329434113,positive,0.6715,NaN,0.0000,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada),1.0
9845,569643648910028801,negative,1.0000,Customer Service Issue,1.0000,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN,0.0
10799,568864981917110272,negative,1.0000,Can't Tell,0.6575,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN,0.0
